# Running Native Functions


Two of the previous notebooks showed how to [execute semantic functions inline](./03-semantic-function-inline.ipynb) and how to [run prompts from a file](./02-running-prompts-from-file.ipynb).

In this notebook, we'll show how to use native functions from a file. We will also show how to call semantic functions from native functions.

This can be useful in a few scenarios:

- Writing logic around how to run a prompt that changes the prompt's outcome.
- Using external data sources to gather data to concatenate into your prompt.
- Validating user input data prior to sending it to the LLM prompt.

Native functions are defined using standard Python code. The structure is simple, but not well documented at this point.

The following examples are intended to help guide new users towards successful native & semantic function use with the SK Python framework.


Prepare a semantic kernel instance first, loading also the AI service settings defined in the [Setup notebook](00-getting-started.ipynb):


Initial configuration for the notebook to run properly.

In [25]:
# Make sure paths are correct for the imports

import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

We will load our settings and get the LLM service to use for the notebook.

In [26]:
from services import Service

from service_settings import ServiceSettings

service_settings = ServiceSettings.create()

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = (
    Service.AzureOpenAI
    if service_settings.global_llm_service is None
    else Service(service_settings.global_llm_service.lower())
)
print(f"Using service type: {selectedService}")

Using service type: Service.AzureOpenAI


We now configure our Chat Completion service on the kernel.

In [27]:
from semantic_kernel import Kernel

kernel = Kernel()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
            env_file_path="../.env",
        ),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
            env_file_path="../.env",
        ),
    )

Let's create a **native** function that gives us a random number between 3 and a user input as the upper limit. We'll use this number to create 3-x paragraphs of text when passed to a semantic function.


First, let's create our native function.


In [28]:
import random

from semantic_kernel.functions import kernel_function


class GenerateNumberPlugin:
    """
    Description: Generate a number between 3-x.
    """

    @kernel_function(
        description="Generate a random number between 3-x",
        name="GenerateNumberThreeOrHigher",
    )
    def generate_number_three_or_higher(self, input: str) -> str:
        """
        Generate a number between 3-<input>
        Example:
            "8" => rand(3,8)
        Args:
            input -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            return str(random.randint(3, int(input)))
        except ValueError as e:
            print(f"Invalid input {input}")
            raise e

Next, let's create a semantic function that accepts a number as `{{$input}}` and generates that number of paragraphs about two Corgis on an adventure. `$input` is a default variable semantic functions can use.


In [29]:
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings, OpenAIChatPromptExecutionSettings
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig

prompt = """
모험을 떠나는 두 웰시코기들의 짧은 이야기를 작성하세요.
스토리가 있어야 합니다:
- G 등급
- 긍정적인 메시지가 있어야 합니다.
- 성차별, 인종차별 또는 기타 편견/편견이 없어야 합니다.
- 정확히 {{$input}} 단락 길이여야 합니다. 이 길이여야 합니다.
"""

if selectedService == Service.OpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = AzureChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-35-turbo",
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="story",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True),
    ],
    execution_settings=execution_settings,
)

corgi_story = kernel.add_function(
    function_name="CorgiStory",
    plugin_name="CorgiPlugin",
    prompt_template_config=prompt_template_config,
)

generate_number_plugin = kernel.add_plugin(GenerateNumberPlugin(), "GenerateNumberPlugin")

In [30]:
# Run the number generator
generate_number_three_or_higher = generate_number_plugin["GenerateNumberThreeOrHigher"]

number_result = await generate_number_three_or_higher(kernel, input=6)
print(number_result)

6


In [31]:
story = await corgi_story.invoke(kernel, input=number_result.value)

_Note: depending on which model you're using, it may not respond with the proper number of paragraphs._


In [32]:
print(f"Generating a corgi story exactly {number_result.value} paragraphs long.")
print("=====================================================")
print(story)

Generating a corgi story exactly 6 paragraphs long.
옛날 옛적에, 푸른 들판과 아름다운 숲으로 둘러싸인 작은 마을에 두 마리의 웰시코기 강아지, 루니와 벤지가 살고 있었습니다. 루니는 항상 호기심이 많고 새로운 것을 탐험하기 좋아하는 성격이었고, 벤지는 조용하고 신중한 성격이었지만 루니를 따라가며 모험을 즐겼습니다. 그들은 마을에서 가장 친한 친구로 알려져 있었고, 항상 함께 다니며 서로를 지켜주었습니다.

어느 날, 루니와 벤지는 마을 외곽에 있는 신비로운 숲으로 모험을 떠나기로 결심했습니다. 그들은 숲 속 깊숙이 들어가면서 다양한 동물 친구들을 만났습니다. 다람쥐 찰리는 그들에게 가장 맛있는 도토리가 어디에 있는지 알려주었고, 부엉이 올리는 길을 잃지 않도록 높은 나무에서 길을 안내해 주었습니다. 두 강아지는 새로운 친구들을 만나고 도움을 받으며 숲 속을 탐험하는 것이 너무나 즐거웠습니다.

숲 속을 걷던 중, 루니와 벤지는 길을 잃고 말았습니다. 벤지는 걱정이 되었지만, 루니는 벤지를 안심시키며 함께 해결책을 찾자고 말했습니다. 그들은 서로를 믿고 협력하며 숲 속에서 빠져나갈 방법을 고민하기 시작했습니다. 두 강아지는 각자의 장점을 살려 문제를 해결하기로 했습니다. 루니는 높은 곳에 올라가 주변을 살펴보고, 벤지는 냄새를 맡아 길을 찾기 시작했습니다.

마침내, 벤지는 마을에서 나는 익숙한 냄새를 맡고 그 방향으로 걸음을 옮기기 시작했습니다. 루니는 벤지를 따라가며 조용히 길을 잃지 않도록 신경 썼습니다. 그들은 서로의 노력을 격려하며 조금씩 마을로 돌아가는 길을 찾아갔습니다. 결국 두 강아지는 힘을 합쳐 무사히 마을로 돌아올 수 있었습니다.

마을로 돌아온 루니와 벤지는 자신들이 겪은 모험 이야기를 마을 친구들에게 들려주었습니다. 그들은 서로를 믿고 협력하면 어떤 어려움도 이겨낼 수 있다는 중요한 교훈을 배웠습니다. 마을 친구들은 두 강아지의 이야기를 들으며 큰 박수를 쳤고, 루니와 벤지는 자신들의 모험이 다른 

## Kernel Functions with Annotated Parameters

That works! But let's expand on our example to make it more generic.

For the native function, we'll introduce the lower limit variable. This means that a user will input two numbers and the number generator function will pick a number between the first and second input.

We'll make use of the Python's `Annotated` class to hold these variables.


In [33]:
kernel.remove_all_services()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
        ),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
        ),
    )

Let's start with the native function. Notice that we're add the `@kernel_function` decorator that holds the name of the function as well as an optional description. The input parameters are configured as part of the function's signature, and we use the `Annotated` type to specify the required input arguments.


In [34]:
import sys
from typing import Annotated

from semantic_kernel.functions import kernel_function


class GenerateNumberPlugin:
    """
    Description: Generate a number between a min and a max.
    """

    @kernel_function(
        name="GenerateNumber",
        description="Generate a random number between min and max",
    )
    def generate_number(
        self,
        min: Annotated[int, "the minimum number of paragraphs"],
        max: Annotated[int, "the maximum number of paragraphs"] = 10,
    ) -> Annotated[int, "the output is a number"]:
        """
        Generate a number between min-max
        Example:
            min="4" max="10" => rand(4,8)
        Args:
            min -- The lower limit for the random number generation
            max -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            return str(random.randint(min, max))
        except ValueError as e:
            print(f"Invalid input {min} and {max}")
            raise e

In [35]:
generate_number_plugin = kernel.add_plugin(GenerateNumberPlugin(), "GenerateNumberPlugin")
generate_number = generate_number_plugin["GenerateNumber"]
result = await generate_number(kernel, min=3, max=8)
print (result)

6


Now let's also allow the semantic function to take in additional arguments. In this case, we're going to allow the our CorgiStory function to be written in a specified language. We'll need to provide a `paragraph_count` and a `language`.


In [36]:
prompt = """
모험을 떠나는 두 웰시코기들의 짧은 이야기를 작성하세요.
스토리가 있어야 합니다:
- G 등급
- 긍정적인 메시지가 있어야 합니다.
- 성차별, 인종차별 또는 기타 편견/편견이 없어야 합니다.
- 정확히 {{$parameter_count}} 문단 길이여야 합니다.
- 이 언어로 작성해야 합니다: {{$language}}
"""

if selectedService == Service.OpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = AzureChatPromptExecutionSettings(
        service_id=service_id,
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="summarize",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="paragraph_count", description="The number of paragraphs", is_required=True),
        InputVariable(name="language", description="The language of the story", is_required=True),
    ],
    execution_settings=execution_settings,
)

corgi_story = kernel.add_function(
    function_name="CorgiStory",
    plugin_name="CorgiPlugin",
    prompt_template_config=prompt_template_config,
)

Let's generate a paragraph count.


In [37]:
result = await generate_number.invoke(kernel, min=1, max=5)
num_paragraphs = result.value
print(f"Generating a corgi story {num_paragraphs} paragraphs long.")

Generating a corgi story 5 paragraphs long.


We can now invoke our corgi_story function using the `kernel` and the keyword arguments `paragraph_count` and `language`.


In [38]:
# Pass the output to the semantic story function
desired_language = "Japanese"
story = await corgi_story.invoke(kernel, paragraph_count=num_paragraphs, language=desired_language)

Variable `Symbols.VAR_PREFIX: parameter_count` not found in the KernelArguments


In [39]:
print(f"Generating a corgi story {num_paragraphs} paragraphs long in {desired_language}.")
print("=====================================================")
print(story)

Generating a corgi story 5 paragraphs long in Japanese.
ある晴れた日の朝、二匹のウェルシュコーギー、ポポとモモは大冒険に出かけることにしました。彼らはいつも一緒に遊んでいましたが、今日は特別な日です。ポポとモモは近くの森に向かい、そこで新しい友達を見つけることが目的でした。森の中で彼らは迷子になった小さなリス、リリーを発見しました。リリーは家に帰る道がわからず、心細そうにしていました。ポポとモモは力を合わせてリリーを元気づけ、彼女の家まで案内しました。リリーは感謝の気持ちでいっぱいになり、ポポとモモに美味しいどんぐりをプレゼントしました。この冒険を通じて、ポポとモモは助け合うことの大切さと、新しい友達を作る喜びを学びました。彼らは幸せな気持ちでいっぱいになり、また新たな冒険を夢見ながら森を後にしました。


## Calling Native Functions within a Semantic Function

One neat thing about the Semantic Kernel is that you can also call native functions from within Prompt Functions!

We will make our CorgiStory semantic function call a native function `GenerateNames` which will return names for our Corgi characters.

We do this using the syntax `{{plugin_name.function_name}}`. You can read more about our prompte templating syntax [here](../../../docs/PROMPT_TEMPLATE_LANGUAGE.md).


In [40]:
from semantic_kernel.functions import kernel_function


class GenerateNamesPlugin:
    """
    Description: Generate character names.
    """

    # The default function name will be the name of the function itself, however you can override this
    # by setting the name=<name override> in the @kernel_function decorator. In this case, we're using
    # the same name as the function name for simplicity.
    @kernel_function(description="Generate character names", name="generate_names")
    def generate_names(self) -> str:
        """
        Generate two names.
        Returns:
            str
        """
        names = {"Hoagie", "Hamilton", "Bacon", "Pizza", "Boots", "Shorts", "Tuna"}
        first_name = random.choice(list(names))
        names.remove(first_name)
        second_name = random.choice(list(names))
        return f"{first_name}, {second_name}"

In [41]:
generate_names_plugin = kernel.add_plugin(GenerateNamesPlugin(), plugin_name="GenerateNames")
generate_names = generate_names_plugin["generate_names"]

In [42]:
prompt = """
모험을 떠나는 두 웰시코기들의 짧은 이야기를 작성하세요.
스토리가 있어야 합니다:
- G 등급
- 긍정적인 메시지가 있어야 합니다.
- 성차별, 인종차별 또는 기타 편견/편견이 없어야 합니다.
- 정확히 {{$parameter_count}} 문단 길이여야 합니다.
- 이 언어로 작성해야 합니다: {{$language}}
- 코기의 두 가지 이름은 {{GenerateNames.generate_names}}입니다.
"""

In [43]:
if selectedService == Service.OpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = AzureChatPromptExecutionSettings(
        service_id=service_id,
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="corgi-new",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="paragraph_count", description="The number of paragraphs", is_required=True),
        InputVariable(name="language", description="The language of the story", is_required=True),
    ],
    execution_settings=execution_settings,
)

corgi_story = kernel.add_function(
    function_name="CorgiStoryUpdated",
    plugin_name="CorgiPluginUpdated",
    prompt_template_config=prompt_template_config,
)

In [44]:
result = await generate_number.invoke(kernel, min=1, max=5)
num_paragraphs = result.value

In [45]:
desired_language = "English"
story = await corgi_story.invoke(kernel, paragraph_count=num_paragraphs, language=desired_language)

Variable `Symbols.VAR_PREFIX: parameter_count` not found in the KernelArguments


In [46]:
print(f"Generating a corgi story {num_paragraphs} paragraphs long in {desired_language}.")
print("=====================================================")
print(story)

Generating a corgi story 1 paragraphs long in English.
Once upon a time, two adventurous Welsh Corgis named Tuna and Bacon set out on a grand journey through the enchanted forest near their home. They were best friends and always stayed by each other’s side. The forest was filled with tall trees, sparkling streams, and friendly woodland creatures. As they traveled, they came across a baby bird that had fallen from its nest. Tuna and Bacon worked together, using their keen senses and gentle paws, to carefully place the baby bird back into its nest. The mother bird chirped joyfully, and the two Corgis felt an immense sense of accomplishment. They continued their journey, helping anyone in need, and spreading kindness wherever they went. By the end of their adventure, Tuna and Bacon learned that the greatest treasure of all is the joy that comes from helping others. They returned home with their hearts full, ready for their next adventure, knowing that together, they could overcome any ch

### Recap

A quick review of what we've learned here:

- We've learned how to create native and prompt functions and register them to the kernel
- We've seen how we can use Kernel Arguments to pass in more custom variables into our prompt
- We've seen how we can call native functions within a prompt.
